# 배열 필드
- 동일한 데이터 유형의 정렬된 요소 집합을 저장

## 제한
- 기본값: ARRAY 필드는 기본값을 지원하지 않음, nullable 속성을 True로 설정하여 null 값을 허용
- 데이터 유형: 배열 필드의 모든 요소는 element_type에 지정된 대로 동일한 데이터 유형을 가져야 함, element_type을 VARCHAR로 설정할 경우 배열 요소에 대해서도 max_length를 설정해야 함
- 배열 용량: 배열 필드의 요소 수는 max_capacity에 지정된 대로 배열을 생성할 때 정의된 최대 용량보다 작거나 같아야함 1 ~ 4096 사이
- 문자열 처리: 배열 필드의 문자열 값은 시맨틱 이스케이프나 변환 없이 있는 그대로 저장, 예를 들어 'a"b', "a'b", 'a\'b', "a\"b" 은 입력한 대로 저장되고 'a'b' 및 "a"b" 은 유효하지 않은 값으로 간주

## ARRAY 필드 추가
- 배열 필드 Milvus를 사용하려면 컬렉션 스키마를 만들 때 관련 필드 유형을 정의
1. datatype을 지원되는 배열 데이터 유형인 ARRAY로 설정
2. element_type 매개변수를 사용하여 배열에 있는 요소의 데이터 유형을 지정, VARCHAR 또는 INT64과 같이 Milvus에서 지원하는 모든 스칼라 데이터 유형이 될 수 있음 -> 동일한 배열의 모든 요소는 동일한 데이터 유형이어야 함
3. max_capacity 매개변수를 사용하여 배열의 최대 용량, 즉 배열에 포함할 수 있는 최대 요소 수를 정의

In [ ]:
from pymilvus import MilvusClient, DataType

SERVER_ADDR = "http://localhost:19530"

client = MilvusClient(uri=SERVER_ADDR)

schema = client.create_schema(
    auto_id=False,
    enable_dynamic_fields=True,
)

schema.add_field(field_name="tags", datatype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=10, max_length=65535, nullable=True)
schema.add_field(field_name="ratings", datatype=DataType.ARRAY, element_type=DataType.INT64, max_capacity=5, nullable=True)
schema.add_field(field_name="pk", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=3)

{'auto_id': False, 'description': '', 'fields': [{'name': 'tags', 'description': '', 'type': <DataType.ARRAY: 22>, 'params': {'max_length': 65535, 'max_capacity': 10}, 'nullable': True, 'element_type': <DataType.VARCHAR: 21>}, {'name': 'ratings', 'description': '', 'type': <DataType.ARRAY: 22>, 'params': {'max_capacity': 5}, 'nullable': True, 'element_type': <DataType.INT64: 5>}, {'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 3}}], 'enable_dynamic_field': False}

## 인덱스 매개변수 설정

In [2]:
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="tags",
    index_type="AUTOINDEX",
    index_name="tags_index"
)

index_params.add_index(
    field_name="embedding",
    index_type="AUTOINDEX",
    metric_type="COSINE"
)

## 컬렉션 만들기

In [3]:
if client.has_collection("my_collection"):
    client.drop_collection("my_collection")

client.create_collection(
    collection_name="my_collection",
    schema=schema,
    index_params=index_params
)

## 데이터 삽입

In [4]:
# Sample data
data = [
  {
      "tags": ["pop", "rock", "classic"],
      "ratings": [5, 4, 3],
      "pk": 1,
      "embedding": [0.12, 0.34, 0.56]
  },
  {
      "tags": None,  # Entire ARRAY is null
      "ratings": [4, 5],
      "pk": 2,
      "embedding": [0.78, 0.91, 0.23]
  },
  {  # The tags field is completely missing
      "ratings": [9, 5],
      "pk": 3,
      "embedding": [0.18, 0.11, 0.23]
  }
]

client.insert(
    collection_name="my_collection",
    data=data
)

{'insert_count': 3, 'ids': [1, 2, 3], 'cost': 0}

## 필터 표현식을 사용한 쿼리

In [5]:
filter = 'tags IS NOT NULL'

res = client.query(
    collection_name="my_collection",
    filter=filter,
    output_fields=["tags", "ratings", "pk"]
)

print(res)

data: ["{'ratings': [5, 4, 3], 'pk': 1, 'tags': ['pop', 'rock', 'classic']}"]


In [6]:
filter = 'ratings[0] > 4'

res = client.query(
    collection_name="my_collection",
    filter=filter,
    output_fields=["tags", "ratings", "embedding"]
)

print(res)

data: ["{'tags': ['pop', 'rock', 'classic'], 'ratings': [5, 4, 3], 'embedding': [np.float32(0.12), np.float32(0.34), np.float32(0.56)], 'pk': 1}", "{'tags': None, 'ratings': [9, 5], 'embedding': [np.float32(0.18), np.float32(0.11), np.float32(0.23)], 'pk': 3}"]


## 필터 표현식을 사용한 벡터 검색

In [8]:
filter = 'tags[0] == "pop"'

res = client.search(
    collection_name="my_collection",
    data=[[0.3, -0.6, 0.1]],
    limit=5,
    search_params={"params": {"nprobe": 10}},
    output_fields=["tags", "ratings", "embedding"],
    filter=filter
)

print(res)

data: [[{'pk': 1, 'distance': -0.2479381263256073, 'entity': {'tags': ['pop', 'rock', 'classic'], 'ratings': [5, 4, 3], 'embedding': [0.11999999731779099, 0.3400000035762787, 0.5600000023841858]}}]]
